## 🔧 步骤1: 环境配置

首先配置必要的环境和API密钥

In [17]:
import os
import sys
from pathlib import Path

# 添加项目路径
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# 配置API密钥（请替换为你的密钥）
# 方式1: 直接设置（不推荐，仅用于测试）
# os.environ['GEMINI_API_KEY'] = 'your_api_key_here'

# 方式2: 从.env文件读取（推荐）
from dotenv import load_dotenv
load_dotenv()

# 验证
api_key = os.getenv('GEMINI_API_KEY')
if api_key:
    print("✅ API密钥已配置")
    print(f"   密钥前缀: {api_key[:10]}...")
else:
    print("⚠️  未找到API密钥，将使用模拟模式")
    print("   提示: 在.env文件中设置 GEMINI_API_KEY=your_key")

✅ API密钥已配置
   密钥前缀: AIzaSyCglf...


## 📁 步骤2: 选择目标项目

可以选择:
1. GitHub公开项目（自动克隆）
2. 本地项目路径

In [18]:
from src.context_analyzer import GitHubIntegration

# 方式1: 使用GitHub项目（推荐）
# 示例：本体匹配项目
project_source = "https://github.com/qzc438-research/ontology-llm"

# 方式2: 使用本地项目
# project_source = "/Users/yourname/projects/your-project"

# 克隆或使用项目
print("🔄 准备项目...")
project_path = GitHubIntegration.clone_or_use_repo(project_source)

print(f"\n📂 项目路径: {project_path}")
print(f"   项目名称: {project_path.name}")

# 快速浏览项目结构
python_files = list(project_path.rglob('*.py'))
python_files = [f for f in python_files if '__pycache__' not in str(f) and '.venv' not in str(f)]
print(f"   Python文件: {len(python_files)} 个")

🔄 准备项目...
✅ 项目已存在: /Users/xianhaoliu/github_repos/ontology-llm

📂 项目路径: /Users/xianhaoliu/github_repos/ontology-llm
   项目名称: ontology-llm
   Python文件: 13 个


## 🧠 步骤3: 初始化生成器

创建简单生成器实例，支持上下文增强

In [19]:
from src.simple_generator import SimpleGenerator

# 创建生成器
generator = SimpleGenerator(
    project_path=str(project_path),
    model="gemini-2.5-flash",
    temperature=0.3  # 较低温度保证格式稳定性
)

print("✅ 生成器初始化成功")
print(f"   LLM可用: {generator.llm_available}")
print(f"   上下文增强: {generator.context_enabled}")
print(f"   项目: {generator.project_path.name}")

✅ 生成器初始化成功
   LLM可用: True
   上下文增强: True
   项目: ontology-llm


## 📝 步骤4a: 生成单个问答对（测试）

先生成一个问答对测试效果

In [20]:
import random

# 随机选择一个文件
files = generator.discover_python_files()
test_file = random.choice(files)
rel_path = test_file.relative_to(project_path)

print(f"📄 测试文件: {rel_path}\n")

# 提取代码片段
code_snippet = generator.extract_code_snippet(test_file, length=600)
print("【代码片段预览】")
print(code_snippet[:300] + "...\n")

# 生成问答对
print("🔄 生成问答对...")
qa = generator.generate_qa_pair(
    code_snippet=code_snippet,
    file_path=str(rel_path),
    use_context=True  # 启用上下文增强
)

if qa:
    print("\n✅ 生成成功！\n")
    print("="*70)
    print(f"问题: {qa['question']}\n")
    print(f"答案: {qa['answer']}\n")
    print("推理步骤:")
    for i, step in enumerate(qa['reasoning_steps'], 1):
        print(f"  {i}. {step}")
    print("="*70)
else:
    print("❌ 生成失败，请检查API配置")

📄 测试文件: om_database_matching.py

【代码片段预览】
syntactic_matches = pd.DataFrame(syntactic_matching)
    syntactic_matches.drop_duplicates(['entity'], inplace=True)
    if len(syntactic_matches) != 0:
        result = syntactic_matches['entity'].head(top_k).values.tolist()
    else:
        result = [null_value_matching]
    print(result)
    ret...

🔄 生成问答对...

✅ 生成成功！

问题: 在`om_database_matching.py`模块中，我们观察到代码片段处理了`syntactic_matches`，并且`lexical`函数被封装为LangChain的`@tool`。结合项目`ontology-llm`的上下文以及`llm_matching`、`llm_om_zero_shot`等核心模块，请详细阐述该系统是如何编排不同粒度的实体匹配策略（例如句法、词法、以及基于LLM的匹配）。特别是，将`lexical`等传统匹配方法封装为`@tool`在整个匹配流程中提供了哪些具体的架构优势，以及系统如何利用这种编排来处理匹配失败或无结果（如`null_value_matching`）的情况以确保鲁棒性？

答案: 在`ontology-llm`项目中，`om_database_matching.py`模块负责处理实体匹配，它显然采用了**多阶段、混合式**的匹配策略，结合了传统规则方法和先进的LLM能力，并通过LangChain的`@tool`机制进行巧妙编排。

**不同粒度实体匹配策略的编排：**

1.  **初期/高效匹配（句法和词法）：**
    *   **句法匹配 (Syntactic Matching):** 代码中展示的`syntactic_matches`处理部分，通常涉及到基于字符串相似度、正则表达式、编辑距离等快速、确定性的方法。它的目标是识别精确或近似的文本匹配，效率高，适用于处理拼写错误、

## 🏗️ 步骤4b: 生成单个设计方案（测试）

In [21]:
# 定义需求
test_requirement = "为项目添加批量处理功能，支持并发处理大量数据"

print(f"📋 测试需求: {test_requirement}\n")

# 生成设计方案
print("🔄 生成设计方案...")
design = generator.generate_design_solution(
    requirement=test_requirement,
    use_context=True
)

if design:
    print("\n✅ 生成成功！\n")
    print("="*70)
    print(f"需求: {design['requirement']}\n")
    print(f"解决方案: {design['solution']}\n")
    print("实施步骤:")
    for i, step in enumerate(design['steps'], 1):
        print(f"  {i}. {step}")
    print("\n需要修改的文件:")
    for item in design['files_to_modify']:
        print(f"  - {item['file']}: {item['reason']}")
    print("="*70)
else:
    print("❌ 生成失败，请检查API配置")

📋 测试需求: 为项目添加批量处理功能，支持并发处理大量数据

🔄 生成设计方案...

✅ 生成成功！

需求: 为项目添加批量处理功能，支持并发处理大量数据

解决方案: 为了为 `ontology-llm` 项目添加批量处理和并发功能，我们将采用基于 `concurrent.futures` 模块的生产者-消费者模型。`run_series_conference` 模块将作为协调器，负责加载所有待处理的数据项（即“任务”），将其切分为可管理的批次（batch），然后将这些批次提交给一个线程池或进程池（`ThreadPoolExecutor` 或 `ProcessPoolExecutor`）进行并发处理。核心的LLM调用逻辑（如 `llm_om_zero_shot` 或 `llm_matching`）将被封装成一个可被并发执行的函数。`util` 模块将提供批处理和并发相关的辅助函数。

这种方法允许我们：
1. **控制并发度**: 通过配置线程/进程池的大小。
2. **优化资源利用**: 在等待LLM API响应时，其他任务可以同时处理。
3. **提高吞吐量**: 显著减少处理大量数据所需的时间。
4. **弹性扩展**: 易于根据硬件资源或API限制调整参数。

实施步骤:
  1. **控制并发度**: 通过配置线程/进程池的大小。
  2. **优化资源利用**: 在等待LLM API响应时，其他任务可以同时处理。
  3. **提高吞吐量**: 显著减少处理大量数据所需的时间。
  4. **弹性扩展**: 易于根据硬件资源或API限制调整参数。
  5. **定义“任务”结构**:
    *   首先，明确一个单一的LLM本体匹配操作（例如，调用 `llm_om_zero_shot` 进行一次匹配）所需的全部输入数据。这可能是一个字典、一个对象或一个元组，包含源本体信息、目标本体信息、LLM提示模板、其他配置参数等。我们将把每个这样的输入单元称为一个“任务”。
    *   *修改文件*: 检查 `llm_om_zero_shot.py` 和 `llm_matching.py` 中核心函数的签名，确保它们能够接收一个封装了所有必要信息的单一“任务”对象作为输入。
  6. **创建数据加载和任务生成模块**:
    *   在 `gene

## 🚀 步骤5: 批量生成训练数据

一键生成完整训练数据集

In [22]:
# 配置生成参数
NUM_QA_PAIRS = 10      # 问答对数量
NUM_DESIGN = 5         # 设计方案数量
USE_CONTEXT = True     # 是否启用上下文增强

print("="*70)
print("🎯 批量生成配置")
print("="*70)
print(f"   问答对数量: {NUM_QA_PAIRS}")
print(f"   设计方案数量: {NUM_DESIGN}")
print(f"   上下文增强: {USE_CONTEXT}")
print(f"   预计耗时: {(NUM_QA_PAIRS + NUM_DESIGN) * 15 / 60:.1f} 分钟\n")

# 执行批量生成
dataset = generator.generate_batch(
    num_qa=NUM_QA_PAIRS,
    num_design=NUM_DESIGN,
    use_context=USE_CONTEXT
)

print("\n🎉 批量生成完成！")

🎯 批量生成配置
   问答对数量: 10
   设计方案数量: 5
   上下文增强: True
   预计耗时: 3.8 分钟

🚀 开始生成训练数据

📁 发现Python文件...
   找到 13 个文件

📝 生成 10 个问答对...
   [1/10] 处理文件: generate_anatomy_mse_benchmark.py
       ✅ 成功
   [2/10] 处理文件: generate_anatomy_mse_benchmark.py
       ✅ 成功
   [3/10] 处理文件: run_series_similarity.py
       ✅ 成功
   [4/10] 处理文件: run_config.py
       ✅ 成功
   [5/10] 处理文件: generate_conference_benchmark.py
       ✅ 成功
   [6/10] 处理文件: run_series_conference.py
       ✅ 成功
   [7/10] 处理文件: run_series_conference.py
       ✅ 成功
   [8/10] 处理文件: run_series_similarity.py
       ✅ 成功
   [9/10] 处理文件: llm_om_zero_shot.py
       ✅ 成功
   [10/10] 处理文件: om_csv_to_database.py
       ✅ 成功

🏗️  生成 5 个设计方案...
   [1/5] 需求: 扩展run_series_conference以支持弱网环境场景...
       ✅ 成功
   [2/5] 需求: 扩展util以支持微服务场景...
       ✅ 成功
   [3/5] 需求: 优化generate_conference_benchmark的可维护性性能...
       ✅ 成功
   [4/5] 需求: 改进llm_om_zero_shot的开发者体验...
       ✅ 成功
   [5/5] 需求: 重构util以支持热更新...
       ✅ 成功

📊 生成完成
   问答对: 10/10
   设计方案: 5/5
   成功率: 100.0%

🎉 

## 📊 步骤6: 数据质量分析

分析生成数据的质量指标

In [23]:
import json
from collections import Counter

# 统计问答对
print("="*70)
print("📊 问答对质量分析")
print("="*70)

if dataset['qa_pairs']:
    # 基础统计
    total_qa = len(dataset['qa_pairs'])
    avg_question_len = sum(len(qa['question']) for qa in dataset['qa_pairs']) / total_qa
    avg_answer_len = sum(len(qa['answer']) for qa in dataset['qa_pairs']) / total_qa
    avg_reasoning_steps = sum(len(qa['reasoning_steps']) for qa in dataset['qa_pairs']) / total_qa
    
    print(f"   总数量: {total_qa}")
    print(f"   平均问题长度: {avg_question_len:.0f} 字符")
    print(f"   平均答案长度: {avg_answer_len:.0f} 字符")
    print(f"   平均推理步骤: {avg_reasoning_steps:.1f} 步")
    
    # 文件分布
    source_files = [qa.get('source_file', 'unknown') for qa in dataset['qa_pairs']]
    file_dist = Counter(source_files)
    print(f"\n   文件覆盖数: {len(file_dist)}")
    print(f"   文件分布: {dict(list(file_dist.most_common(3)))}")
    
    # 示例展示
    print("\n   【示例问答对】")
    sample = dataset['qa_pairs'][0]
    print(f"   Q: {sample['question'][:80]}...")
    print(f"   A: {sample['answer'][:80]}...")
else:
    print("   ⚠️  未生成问答对")

# 统计设计方案
print("\n" + "="*70)
print("🏗️  设计方案质量分析")
print("="*70)

if dataset['design_solutions']:
    total_design = len(dataset['design_solutions'])
    avg_steps = sum(len(d['steps']) for d in dataset['design_solutions']) / total_design
    avg_files = sum(len(d['files_to_modify']) for d in dataset['design_solutions']) / total_design
    
    print(f"   总数量: {total_design}")
    print(f"   平均实施步骤: {avg_steps:.1f} 步")
    print(f"   平均修改文件: {avg_files:.1f} 个")
    
    # 示例展示
    print("\n   【示例设计方案】")
    sample = dataset['design_solutions'][0]
    print(f"   需求: {sample['requirement'][:60]}...")
    print(f"   方案: {sample['solution'][:60]}...")
else:
    print("   ⚠️  未生成设计方案")

print("\n" + "="*70)

📊 问答对质量分析
   总数量: 10
   平均问题长度: 575 字符
   平均答案长度: 4396 字符
   平均推理步骤: 11.4 步

   文件覆盖数: 7
   文件分布: {'generate_anatomy_mse_benchmark.py': 2, 'run_series_similarity.py': 2, 'run_series_conference.py': 2}

   【示例问答对】
   Q: The provided code snippet demonstrates a series of benchmarking steps for ontolo...
   A: This code snippet reveals a robust and iterative evaluation strategy for the `on...

🏗️  设计方案质量分析
   总数量: 5
   平均实施步骤: 6.8 步
   平均修改文件: 8.4 个

   【示例设计方案】
   需求: 扩展run_series_conference以支持弱网环境场景...
   方案: 为 `run_series_conference` 引入弹性机制，以应对弱网环境下的网络延迟、连接中断和请求超时等问题。...



## 💾 步骤7: 保存训练数据

将数据保存为JSON格式

In [24]:
from datetime import datetime

# 生成输出文件名
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
project_name = project_path.name
output_file = f"outputs/{project_name}/training_data_{timestamp}.json"

# 保存数据
generator.save_dataset(dataset, output_file)

# 验证文件
output_path = Path(output_file)
if output_path.exists():
    file_size = output_path.stat().st_size
    print(f"\n✅ 文件已保存")
    print(f"   路径: {output_path}")
    print(f"   大小: {file_size / 1024:.1f} KB")
    
    # 读取验证
    with open(output_path, 'r', encoding='utf-8') as f:
        loaded = json.load(f)
    print(f"   问答对: {len(loaded['qa_pairs'])}")
    print(f"   设计方案: {len(loaded['design_solutions'])}")


💾 数据已保存: outputs/ontology-llm/training_data_20251219_162158.json
   文件大小: 174.8 KB

✅ 文件已保存
   路径: outputs/ontology-llm/training_data_20251219_162158.json
   大小: 174.8 KB
   问答对: 10
   设计方案: 5


## 🎓 步骤8: 数据预览

查看完整的训练数据样本

In [25]:
import json

# 美化输出第一个问答对
if dataset['qa_pairs']:
    print("="*70)
    print("📝 完整问答对示例")
    print("="*70)
    sample_qa = dataset['qa_pairs'][0]
    print(json.dumps(sample_qa, ensure_ascii=False, indent=2))
    print("\n")

# 美化输出第一个设计方案
if dataset['design_solutions']:
    print("="*70)
    print("🏗️  完整设计方案示例")
    print("="*70)
    sample_design = dataset['design_solutions'][0]
    print(json.dumps(sample_design, ensure_ascii=False, indent=2))

📝 完整问答对示例
{
  "question": "The provided code snippet demonstrates a series of benchmarking steps for ontology alignment across different years (2022 and 2023) using distinct file paths and functions like `find_alignment` and `util.calculate_benchmark_metrics`. Given the broader context of an `ontology-llm` project, what does this specific structure of benchmarking reveal about the evaluation strategy for LLM-based ontology alignment? Furthermore, how do the varying file paths and the sequential nature of these calls (e.g., 2022 vs. 2023 data) contribute to assessing the maturity or iterative improvement of the `llm_matching` or `llm_om_zero_shot` components over time, especially considering potential changes in benchmark datasets or alignment algorithms?",
  "answer": "This code snippet reveals a robust and iterative evaluation strategy for the `ontology-llm` project, designed to systematically benchmark the performance and track the evolution of LLM-based ontology alignment techniques

## 🚀 快捷方式: 一键生成

如果不需要详细步骤，可以使用快捷函数

In [26]:
from src.simple_generator import quick_generate

# 一键生成（所有步骤自动完成）
dataset_quick = quick_generate(
    project_path=str(project_path),
    num_qa=5,
    num_design=3,
    output_path=f"outputs/{project_name}/quick_output.json",
    use_context=True
)

print("\n🎉 快捷生成完成！")

🚀 开始生成训练数据

📁 发现Python文件...
   找到 13 个文件

📝 生成 5 个问答对...
   [1/5] 处理文件: run_series_conference.py
       ✅ 成功
   [2/5] 处理文件: generate_conference_benchmark.py
       ✅ 成功
   [3/5] 处理文件: util.py
       ✅ 成功
   [4/5] 处理文件: generate_conference_benchmark.py
       ✅ 成功
   [5/5] 处理文件: llm_om_few_shot.py
       ✅ 成功

🏗️  生成 3 个设计方案...
   [1/3] 需求: 为generate_anatomy_mse_benchmark添加单元测试保障机制...
       ✅ 成功
   [2/3] 需求: 优化llm_om_few_shot的并发能力性能...
       ✅ 成功
   [3/3] 需求: 扩展om_csv_to_database以支持弱网环境场景...
       ✅ 成功

📊 生成完成
   问答对: 5/5
   设计方案: 3/3
   成功率: 100.0%

💾 数据已保存: outputs/ontology-llm/quick_output.json
   文件大小: 94.3 KB

🎉 快捷生成完成！


---

## 📚 附录: 进阶配置


### 常见问题

**Q: API调用失败怎么办？**
- 检查API密钥是否正确
- 验证网络连接
- 查看API配额限制

**Q: 如何提高生成质量？**
- 启用上下文增强 (`use_context=True`)
- 选择合适的上下文级别
- 调整temperature参数（0.2-0.4最佳）

**Q: 如何加速生成？**
- 使用`minimal`上下文级别
- 减少单次生成数量
- 使用更快的模型